In [1]:
#!pip3 install selenium
#!pip3 install tqdm
#!pip3 install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

In [2]:
import pandas as pd
from datetime import datetime
from datetime import date 

from time import sleep
from tqdm.notebook import tqdm, trange

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

## Bitso Historical Live Scraping

In [3]:
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']
URLs={}
for i in coins:
    URLs.update({i:'https://bitso.com/trade/market/'+i+'/mxn'})

In [53]:
def readscreen(test=False, verbose=False, database=[],order='asc',limitedStop=False, stopLimit=0):
    try:
        # Getting candels tags
        tagsPath = driver.find_elements_by_tag_name('g')
        candels=[]
        for i in range(len(tagsPath)):
            if tagsPath[i].get_attribute("aria-label") != None and tagsPath[i].get_attribute("aria-label").find('Price')==0:
                #print(tagsPath[i].get_attribute("transform"), tagsPath[i].get_attribute("aria-label"))
                candels.append(tagsPath[i])
        print("\n",len(candels), 'Candels detected')

        #Checking every candel by moving arround
        nCandels = len(candels)
        
        if limitedStop==False:
            steps=nCandels
        else:
            steps=stopLimit
        
        for i in trange(steps):
            try:
                action=ActionChains(driver)
                if order == 'asc':
                    action.move_to_element(candels[i])
                elif order == 'des':
                    action.move_to_element(candels[nCandels-i-1])
                else:
                    print('ERROR: Non order detected, use asc or des')
                
                action.perform()
                #print(i, candels[i].get_attribute('aria-label'))
                
                #Getting info of every candel

                voltag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div/div/div')
                volume=voltag.text

                boxtags = driver.find_elements_by_tag_name('b')
                i_open=boxtags[0].text
                i_low=boxtags[1].text
                i_high=boxtags[2].text
                i_close=boxtags[3].text

                datetag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[1]/div/div/div[2]/div')
                date_str=datetag.text
                i_date=datetime.strptime(date_str,'%b %d, %Y')

                day=[i_date,i_open,i_low,i_high,i_close,volume] #Register integration
                
                if test == False:
                    database.append(day)

                if verbose == True:
                    print(day)
            except:
                print('The Candel',i,'Fails when was tried to read it')
        return day[0].date() #Control date to finish the scanning of windows dates
    except:
            print('ERROR: Mayor Fail - Check the driver is initialized correctly')
   

In [27]:
def getScrollerPosition(si,sf):
    xi,yi=int(si[si.find('(')+1:si.find(',')]),int(si[si.find(',')+1:si.find(')')])
    xf,yf=int(sf[sf.find('(')+1:sf.find(',')]),int(sf[sf.find(',')+1:sf.find(')')])
    return xi,yi,xf,yf

In [6]:
def getScrollers():
    #Getting the tags of the  scrollers for time periods
    loaded=False
    while loaded == False:
        try:
            scroll_tags = driver.find_elements_by_tag_name('g') #The scrollers are over the tag 'g'
            scrollers=[]

            for i in range(len(scroll_tags)):
                role = scroll_tags[i].get_attribute("role") # Scrollers has the exlusive attribute 'role'
                if role != None:
                    #print(scrollers[i].get_attribute("role"), scrollers[i].get_attribute("transform"))
                    scrollers.append(scroll_tags[i])
            loaded=True
            if len(scrollers) == 2:
                print('Scroller elements load successfully')
            else:
                print('Something went wrong with the scrollers')
        except:
            print('Loading scrollers...')
    return scrollers

In [63]:
def scrap(zeroi=-17, zerof=38, max_tries=3,offseti=1 ,offsetf=1,driver=0, wait_loading_time=0,verbose_rs=False, database_rs=[]):
    
    sleep(wait_loading_time)
    #Checking where are the scrollers
    si=scrollers[0].get_attribute("transform")
    sf=scrollers[1].get_attribute("transform")

    xi,yi,xf,yf=getScrollerPosition(si,sf)

    step=zerof-zeroi

    it=0
    tries=0
    loaded= False
    
    # Movig the scrollers for time periods
    
    try:
        while loaded == False:
            print('Fitting the area to scan', end="")
            while (xi != zeroi+step*it or xf != zerof+step*it) and tries < max_tries:
                action=ActionChains(driver)
                action.drag_and_drop_by_offset(scrollers[0],zeroi+step*it-xi-offseti,0)
                action.drag_and_drop_by_offset(scrollers[1],zerof+step*it-xf+offsetf,0)
                action.perform()

                si=scrollers[0].get_attribute("transform")
                sf=scrollers[1].get_attribute("transform")
                xi,yi,xf,yf=getScrollerPosition(si,sf)
                sleep(1)
                tries+=1
                print('.',end="")
            
            print("")
            sleep(1)
            last_day=readscreen(database = database_rs, verbose=verbose_rs) ## Getting info function
            it+=1
            tries=0
            print('Iteracion: ',it)
            if last_day == date.today():
                print('Space completed in ',it,'iterations')
                loaded=True
    except:
        print('Space completed in ',it,'iterations [except]')
    print('Scrap function completed')
    

In [8]:
def save(name,database=[]):
    df=pd.DataFrame(database)
    df=df.drop_duplicates()
    df=df.reset_index()
    df.rename(columns={0:'date',1:'open',2:'low',3:'high',4:'close',5:'volume'},inplace=True)
    df=df.drop(columns='index')
    df.to_csv('./'+name+'-mxn.csv',index=False)

In [76]:
def loadWebpage(URL=''):
    print('Opening browser...')
    driver = webdriver.Firefox()
    print('Opening ',URL)
    driver.get(URL)
    print('Maximizing window...')
    driver.maximize_window()
    print('Scrolling into graph...')
    driver.execute_script("window.scrollTo(0, 200)")
    return driver

## Scrapper

In [10]:
for i in coins:
    database=[]
    driver = loadWebpage(URLs[i])
    scrollers = getScrollers()
    scrap(driver=driver,database_rs=database)
    save(i,database)
    print(i, ' Scrapped succesfully')
    driver.close()

Opening browser...
Maximizing window...
Scrolling into graph...
Loading scrollers...
Scroller elements load successfully
Fitting the area to scan...

 111 Candels detected



Iteracion:  1
Fitting the area to scan...

 114 Candels detected


The Candel 113 Fails when was tried to read it

Iteracion:  2
Fitting the area to scan...

 113 Candels detected



Iteracion:  3
Fitting the area to scan...

 114 Candels detected


The Candel 113 Fails when was tried to read it

Iteracion:  4
Fitting the area to scan...

 113 Candels detected


...


Iteracion:  17
Fitting the area to scan...

 4 Candels detected



Iteracion:  18
Fitting the area to scan...

 3 Candels detected



Iteracion:  19
Fitting the area to scan...

 3 Candels detected



Iteracion:  20
Fitting the area to scan...

 3 Candels detected



Iteracion:  21
Space completed in  21 iterations
Scrap function completed
dai  Scrapped succesfully


## Daily Scraper

In [77]:
updates=[]
for i in coins:
    database=[]
    driver = loadWebpage(URLs[i])
    scrollers = getScrollers()
    readscreen(database=database,order='des',limitedStop=True, stopLimit=2)
    updates.append([i,database])
    driver.close()

Opening browser...
Opening  https://bitso.com/trade/market/btc/mxn
Maximizing window...
Scrolling into graph...
Loading scrollers...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/eth/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/xrp/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/ltc/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/bch/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected


...


Opening browser...
Opening  https://bitso.com/trade/market/mana/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/gnt/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/bat/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 92 Candels detected



Opening browser...
Opening  https://bitso.com/trade/market/dai/mxn
Maximizing window...
Scrolling into graph...
Scroller elements load successfully

 60 Candels detected


The Candel 0 Fails when was tried to read it



In [ ]:
#Get Value
#Open Historical
#Add new values (Day after today)
#

In [78]:
updates

[['btc',
  [[datetime.datetime(2020, 6, 1, 0, 0),
    '210,861.46',
    '208,753.02',
    '214,499.99',
    '213,830.52',
    '146.34921297'],
   [datetime.datetime(2020, 5, 31, 0, 0),
    '211,847.01',
    '208,550',
    '213,445.25',
    '210,249.7',
    '84.02252169']]],
 ['eth',
  [[datetime.datetime(2020, 6, 1, 0, 0),
    '5,287.35',
    '5,182.54',
    '5,390',
    '5,389.99',
    '2,591.65311196'],
   [datetime.datetime(2020, 5, 31, 0, 0),
    '5,287.94',
    '5,107.51',
    '5,347.6',
    '5,287.34',
    '1,389.5444322']]],
 ['xrp',
  [[datetime.datetime(2020, 6, 1, 0, 0),
    '4.51',
    '4.44',
    '4.58',
    '4.56',
    '35,823,916.844972'],
   [datetime.datetime(2020, 5, 31, 0, 0),
    '4.52',
    '4.46',
    '4.72',
    '4.52',
    '7,579,455.6677262']]],
 ['ltc',
  [[datetime.datetime(2020, 6, 1, 0, 0),
    '1,017.48',
    '1,005.79',
    '1,048.69',
    '1,044.12',
    '852.77884462'],
   [datetime.datetime(2020, 5, 31, 0, 0),
    '1,030',
    '1,002',
    '1,052.99',
 

In [58]:
driver.close()

## Other Sources

In [ ]:
import requests
from datetime import date

In [ ]:
books={'Bitcoin':'btc_mxn',
       'Ethereum':'eth_mxn',
       'Ripple':'xrp_mxn',
       'LTC':'ltc_mxn',
       'Bitcoin Cash':'bch_mxn',
       'True Dollar':'tusd_mxn',
       'Mana':'mana_mxn',
       'Golem':'gnt_mxn',
       'BAT':'bat_mxn',
       'DAI':'dai_mxn'}

coins=['bitcoin','ethereum','xrp','litecoin','bitcoin-cash','trueusd','decentraland','golem-network-tokens',
       'basic-attention-token','multi-collateral-dai']
start='20120428'

end= date.today()
end= end.strftime("%Y%m%d")

url=[]
for coin in coins:
    url.append('https://coinmarketcap.com/currencies/'+coin+'/historical-data/?start='+start+'&end='+end)

In [ ]:
url[0]